# ESM

In [1]:
import torch

In [2]:
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm1b_t33_650M_UR50S")

Using cache found in /home/t-fli/.cache/torch/hub/facebookresearch_esm_main


In [7]:
aa_emb = model.embed_tokens.weight.detach().cpu().numpy()
aa_emb.shape

(33, 1280)

In [4]:
last_weights = list(model.parameters())[-1]
last_weights, last_weights.shape

(Parameter containing:
 tensor([0.0032, 0.0592, 0.0474,  ..., 0.0142, 0.0754, 0.0690],
        requires_grad=True),
 torch.Size([1280]))

In [5]:
next(model.parameters()).shape

torch.Size([33, 1280])

In [ ]:
len(list(model.parameters()))

In [ ]:
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# Prepare data (first 2 sequences from ESMStructuralSplitDataset superfamily / 4)
data = [
    ("2gi9", "MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"),
]
batch_labels, batch_strs, batch_tokens = batch_converter(data)

# Extract per-residue representations (on CPU)
with torch.no_grad():
    results = model(batch_tokens, repr_layers=[33], return_contacts=True)
token_representations = results["representations"][33]

# Generate per-sequence representations via averaging
# NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
sequence_representations = []
for i, (_, seq) in enumerate(data):
    sequence_representations.append(token_representations[i, 1 : len(seq) + 1].mean(0))

# Look at the unsupervised self-attention map contact predictions
import matplotlib.pyplot as plt
for (_, seq), attention_contacts in zip(data, results["contacts"]):
    plt.matshow(attention_contacts[: len(seq), : len(seq)])
    plt.title(seq)
    plt.show()

In [ ]:
results.keys()

In [ ]:
results["representations"][33].shape